In [44]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from netgen.csg import Pnt
import pyngcore



In [45]:
#help(unit_square)

l = 1
for i in range(1):
    l = l/2

print(l)

netgen_mesh = unit_square.GenerateMesh(maxh=l, segmentsperedge=(1)+1.2, grading=0.1)
mesh = Mesh(netgen_mesh)
#help(mesh)
unref_verts = mesh.vertices
parent_el = mesh.ne
#issues with this-> maskes certain spaces not work
# mesh.Refine()
# mesh.Refine()
# mesh.Refine()

H_h = H1(mesh,order=1)
unref_dofnr = mesh.nv #H_h.ndof #mesh.ne

#help(mesh)

print(unref_dofnr)
Draw(mesh)

0.5
9


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [46]:
#v = mesh.vertices[0]
#help(NodeId)

In [47]:
#reftree = pyngcore.pyngcore.Array_S_S([4,4,3,3,5,5,1,1,1,1,1,1])

#help(mesh.BuildRefinementTree())

#mesh.RefineFromTree(reftree)

#mesh.RefineHP(2,factor=0.135)

In [48]:
netgen_mesh.SplitPowellSabin()
mesh.ngmesh.Update()

mesh._updateBuffers()

H_h.Update() 

# for v in mesh.vertices:
#     print(mesh.GetParentVertices(v.nr))

#might need to be number of faces in barycentrically refined mesh
bary_dofnr = H_h.ndof #mesh.ne
print(bary_dofnr)

#help(H_h)
# print(H_h.Prolongation())
# help(H_h.Prolongation())
Draw(mesh)

# print(mesh.ne)
# for el in mesh.Elements():
#     print(el,el.faces)


33


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [49]:
#H_h = H1(mesh,order=1)
#help(H_h.Prolongation())
# for el in mesh.Elements(BND):
#     print(el.vertices)
#     print(el.edges)
#     print(el.faces)

# print(mesh.Elements(BND).stop -1)

In [50]:
# for el in mesh.Elements():
#     print(el.nr)
    #print(len(H_h.GetDofNrs(el)))
    #print(mesh.GetParentElement(el))
    # for v in el.vertices:
    #     print(v.nr)
    # print(el.edges)
    # print(el.faces)

In [51]:
# dofs_one_element = 3
# element_per_dof = 14 #H_h.ndof//mesh.ne
# # print(netgen_mesh.dim)
# #bary_dofnr//unref_dofnr == 6 (for L2)
# children = np.zeros((unref_dofnr,element_per_dof*dofs_one_element),dtype=np.int64) - 1

# indices = np.zeros(unref_dofnr, dtype=np.int64)

# occurences = np.zeros(bary_dofnr,dtype=np.int8)

# for par in unref_verts:
#     parent = par.nr
#     #print("parent =", parent)
#     for el in par.elements:
#         #print(el.nr)
#         for dofnr in H_h.GetDofNrs(el):
#             #print(dofnr)
#             children[parent,indices[parent]] = dofnr
#             indices[parent] += 1
#             occurences[dofnr] += 1
#         #help(el)
#         #break

# #print(children)


# occurences[0:unref_dofnr] = 1

#print(occurences)
#TODO Count up occurences of dofs in children list (in mapping matrix just increment corresponding entry according to some rule)

In [52]:
#Mapping from refined to coarse mesh


data_length = 0

for par in unref_verts:
    for el in par.elements:
        data_length += len(H_h.GetDofNrs(el))

data = np.zeros(data_length)
row_ind = np.empty(data_length, dtype=int)
col_ind = np.empty(data_length, dtype=int)
shape = (unref_dofnr,bary_dofnr)

j = 0

for par in unref_verts:
    #v = mesh.vertices.__getitem__(i)
    #print(v)
    par_dofnr = H_h.GetDofNrs(par)[0]
    par_nf = len(par.elements)
    for el in par.elements:
        ele = mesh.__getitem__(el)
        for v in ele.vertices:
            ver = mesh.__getitem__(v)
            child_dofnr = H_h.GetDofNrs(v)[0]
            #print(dofnr)
            row_ind[j] = par_dofnr
            col_ind[j] = child_dofnr
            if par_dofnr == child_dofnr:
                data[j] = 1/par_nf
            elif len(ver.elements) == 6:
                data[j] = 1/6
            elif len(ver.elements) == 2:
                data[j] = 1/2
            else:
                data[j] = 1/4
            j += 1

            
# print(data)
# print(row_ind)
# print(col_ind)

# sci_mat = sp.sparse.csr_matrix((data, (row_ind, col_ind)),shape)
# print(sci_mat.todense())
# sci_coo = sci_mat.tocoo()

ng_mat = la.SparseMatrixdouble.CreateFromCOO(row_ind,col_ind,data, unref_dofnr,bary_dofnr)
print(ng_mat.ToDense())

       1       0       0       0       0       0       0       0       0     0.5       0     0.5 0.333333       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0
       0       1       0       0       0       0       0       0       0       0       0       0       0     0.5       0     0.5 0.333333       0       0     0.5       0 0.333333       0       0       0       0       0       0       0       0       0       0       0
       0       0       1       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0     0.5       0     0.5 0.333333       0       0     0.5 0.333333       0       0       0
       0       0       0       1       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0

In [53]:
#H_h = L2(mesh,order=0) 
 


gfu = GridFunction(H_h,multidim = unref_dofnr)



In [54]:
#help(mesh)

# for el in mesh.Elements():    
#     dofnr = H_h.GetDofNrs(el)
#     #print(H_h.GetDofNrs(el.faces[0])) #same result as above as each element only contains 1 face
#     print(dofnr)

In [55]:
print(H_h.ndof)

gfu.Set(0) 

# for i in range(unref_dofnr):
#     for j in children[i,:]:
#         # print(j)
#         if j != -1:
#             if occurences[j] == 1:
#                 gfu.vecs[i][j] = 1.0
#             elif occurences[j] == 2:
#                 gfu.vecs[i][j] = 0.5
#             elif occurences[j] == 4:
#                 gfu.vecs[i][j] = 0.5
#             elif occurences[j] == 6:
#                 gfu.vecs[i][j] = 1/3




#Draw(gfu, animate = True, min = 0, max = 2.0, deformation = False)

33


In [56]:
dfu = GridFunction(H_h,multidim = unref_dofnr)
#help(ng_mat)



dfu.Set(0)
for i in range(unref_dofnr):
    #print(ng_mat.ToDense().NumPy()[:,i])
    dfu.vecs[i][:] = ng_mat.ToDense().NumPy()[i,:]


# for i in range(4):
    #print(dfu.vecs[i][:].FV().NumPy())

Draw(dfu, animate = False, min = 0, max = 2.0, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [57]:
netgen_mesh = unit_square.GenerateMesh(maxh=l, segmentsperedge=(1)+1.2, grading=0.1)

ref_mesh = Mesh(netgen_mesh)

H_h = H1(ref_mesh,order=1)

gfu = GridFunction(H_h,multidim = H_h.ndof)

gfu.Set(0) 

for i in range(H_h.ndof):
    gfu.vecs[i][i] = 1.0

Draw(gfu, animate = False, min = 0, max = 2.0, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene